In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import warnings
import plotly.io as pio
from itertools import chain
from plots import plot_game_play_id
from distances import calc_dist
from shapely import geometry
from sklearn.preprocessing import StandardScaler
from shapely.ops import unary_union
from shapely.validation import make_valid
from shapely.errors import ShapelyDeprecationWarning

In [2]:
pio.renderers.default = 'jupyterlab'
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [3]:
GAMES_PATH='./data/games.csv'
PFF_SCOUTING_DATA_PATH='./data/pffScoutingdata.csv'
PLAYERS_PATH='./data/players.csv'
PLAYS_PATH='./data/plays.csv'
WEEK_PATH='./data/week{}.csv'
WEEK_IDS = [1,2,3,4]
PASS_RESULTS = ['I','C','IN']
SNAP_EVENTS = ['ball_snap', 'autoevent_ballsnap']
NONSNAP_EVENTS = ['autoevent_passforward', 'pass_forward', 'qb_sack', 'autoevent_passinterrupted',
                  'pass_tipped', 'qb_strip_sack', 'autoevent_passinterrupted']
OLINE_POS_lINEDUP= ['C','LT','LG','RT','RG']

In [4]:
def ccworder(A):
    A= A- np.mean(A, 1)[:, None]
    return np.argsort(np.arctan2(A[1, :], A[0, :]))

def gen_snap_df(week_df, snap_events=SNAP_EVENTS):
    snap_df = (week_df
               .query('event.isin(@snap_events)')
               .groupby(['gameId','playId'])
               [['frameId', 'time']].min()
               .reset_index()
               .rename({'frameId': 'snap_frameId',
                        'time': 'snap_time'}, axis=1))
    return snap_df

def gen_nonsnap_df(week_df, nonsnap_events=NONSNAP_EVENTS):
    nonsnap_df = (week_df
               .query('event.isin(@nonsnap_events)')
               .groupby(['gameId','playId'])
               [['frameId','time']].min()
               .reset_index()
               .rename({'frameId': 'nonsnap_frameId',
                        'time': 'nonsnap_time'},axis=1))
    return nonsnap_df

def gen_passer_df(week_df):
    passer_df = (week_df.query('pff_role == "Pass"')
                 [['gameId', 'playId','frameId','time', 'x', 'y', 's','a','dis','o','dir']]
                 .rename({'x': 'qb_x',
                          'y': 'qb_y',
                          's': 'qb_s',
                          'a': 'qb_a',
                          'o': 'qb_o',
                          'dis': 'qb_dis',
                          'dir': 'qb_dir'}, axis=1))
    return passer_df

def gen_line_df(week_df, position, id_cols = ['gameId','nflId', 'playId','frameId'], feats_cols = ['x', 'y', 's','a','dis','o','dir'], oline_pos_lineup=OLINE_POS_lINEDUP, lookback_pd=5, min_speed=0.01, t=0.1):
    
    all_cols = list(chain(*[id_cols, feats_cols]))
    
    rename_cols = {}
    for f in feats_cols:
        renamed_col = f'{position}_{f}'
        rename_cols[f] = renamed_col
    
    if position == 'pb':
        line_df = (week_df
                    .query('pff_role == "Pass Block"'))
                    
        if oline_pos_lineup:
            line_df = (line_df.query('pff_positionLinedUp.isin(@oline_pos_lineup)'))
    else:
        line_df = (week_df
                    .query('pff_role == "Pass Rush"'))
    
    line_df = (line_df[all_cols].rename(rename_cols, axis=1))
    
    line_id_df = line_df[['gameId','playId','nflId']].drop_duplicates().groupby(['gameId', 'playId']).apply(lambda x: x.reset_index(drop=True).reset_index()).reset_index(drop=True).rename({'index': f'{position}_id'}, axis=1)

    line_df = (line_df.merge(line_id_df, how='left', on=['gameId', 'playId', 'nflId']))
    line_df[f'{position}_displacement'] = line_df.apply(lambda x: max(x[f'{position}_s'], min_speed) * t + 0.5*(x[f'{position}_a']) * t**2, axis=1)
    line_df[f'{position}_point'] = line_df.apply(lambda x: geometry.Point(x[f'{position}_x'], x[f'{position}_y']).buffer(x[f'{position}_displacement']), axis=1)
    
    all_movement_paths = []
    for k, df_grp in line_df.groupby(['gameId', 'playId', 'nflId']):
        
        all_paths = []
        for i in range(1, len(df_grp)+1):
            all_paths.append(unary_union(df_grp[f'{position}_point'].values[max(0, i - lookback_pd):i]))
        
        df_grp[f'{position}_movement_path'] = all_paths
        all_movement_paths.append(df_grp)
    line_df_poly = pd.concat(all_movement_paths)
    return line_id_df, line_df_poly

def pivot_line_df(line_df, position):
    line_df = line_df.pivot(index=['gameId', 'playId', 'frameId'], columns=[f'{position}_id'], values=[f'{position}_point', f'{position}_movement_path', f'{position}_x', f'{position}_y', f'{position}_o', f'{position}_dir', f'{position}_s', f'{position}_a', f'{position}_dis'])
    line_df.columns = ['{}_{}'.format(c[0], c[1]) for c in line_df.columns]
    line_df.reset_index(inplace=True)
    return line_df

def gen_pocket_polygon(row):
    
    # row = pocket_area_df.iloc[0]
    
    all_coords = []
    for c in row.index.values:
        
        if 'pb_x' in c:
            if not(pd.isnull(row[c])):
                all_coords.append((row[c], row[c.replace('_x', '_y')]))
    
    all_coords2 = np.array(all_coords)[np.argsort(np.array(all_coords)[:,1])]
    oline_coords2 = np.vstack([all_coords2,[row['qb_x'], row['qb_y']]])
    
    all_coords.append((row['qb_x'], row['qb_y']))
    
    A = np.array(all_coords).T
    sorted_order = ccworder(A)
    oline_coords = A.T[sorted_order]
    
    geom1 = geometry.Polygon(oline_coords)
    geom2 = geometry.Polygon(oline_coords2)
    
    if geom1.area > geom2.area:
        return geom1
    else:
        return geom2
    
def gen_pocket_area_df(passer_df, pass_blockers_df, snap_df, nonsnap_df):
    
    pass_blockers_df = pivot_line_df(pass_blockers_df, position='pb')
    
    pocket_area_df = (passer_df
                          .merge(pass_blockers_df, how='left', on=['gameId', 'playId', 'frameId'])
                          .merge(snap_df, how='left', on=['gameId', 'playId'])
                          .merge(nonsnap_df, how='left', on=['gameId', 'playId'])
                          .query('(frameId >= snap_frameId) & (frameId <= nonsnap_frameId)')
                          .assign(
                              time_since_snap = lambda x: (pd.to_datetime(x['time']) - pd.to_datetime(x['snap_time'])).dt.total_seconds()
                              ))
    
    pocket_area_df['pocket_polygon'] = pocket_area_df.apply(lambda x: gen_pocket_polygon(x), axis=1)
    pocket_area_df['pocket_area'] = pocket_area_df['pocket_polygon'].apply(lambda x: x.area)
    return pocket_area_df

def gen_pocket_adj_risk_poly(row):
    pocket_poly = row['pocket_polygon']
    pocket_poly = make_valid(pocket_poly)
    
    pb_paths = [p for p in row.index.values if 'pb_movement_path' in p]
    pr_paths = [p for p in row.index.values if 'pr_movement_path' in p]
    
    pb_paths_poly = [p for p in row[pb_paths].values if not(pd.isnull(p))]
    pr_paths_poly = [p for p in row[pr_paths].values if not(pd.isnull(p))]
    
    pr_combined = unary_union(pr_paths_poly)
    pb_combined = unary_union(pb_paths_poly)
    
    attack_polys = (pr_combined.symmetric_difference(pb_combined)).difference(pb_combined)
    attack_polys = make_valid(attack_polys)
    
    attack_in_pocket_poly = pocket_poly.intersection(attack_polys)
    
    return attack_in_pocket_poly

def nthHarmonic(N) :
 
    # H1 = 1
    harmonic = 1.00
 
    # loop to apply the formula
    # Hn = H1 + H2 + H3 ... +
    # Hn-1 + Hn-1 + 1/n
    for i in range(2, N + 1) :
        harmonic += 1 / i
 
    return harmonic

## Load the Data

In [5]:
games_df = pd.read_csv(GAMES_PATH)
plays_df = pd.read_csv(PLAYS_PATH)
players_df = pd.read_csv(PLAYERS_PATH)
pff_scouting_df = pd.read_csv(PFF_SCOUTING_DATA_PATH)

games_df = games_df.query('week.isin(@WEEK_IDS)')
plays_df = plays_df.merge(games_df, how='inner', on=['gameId'])
pff_scouting_df = pff_scouting_df.merge(games_df['gameId'], how='inner', on='gameId', validate='m:1')

In [6]:
#Filter for pass plays only
plays_df = (plays_df
            .query('(pff_playAction == 0) & (passResult.isin(@PASS_RESULTS))')
            .query('foulName1.isna()')
            .query('dropBackType == "TRADITIONAL"'))

In [7]:
#Load the weekly data
week_df = []
for week_id in WEEK_IDS:
    week_df.append(pd.read_csv(WEEK_PATH.format(week_id)))
week_df = pd.concat(week_df)

week_df = (week_df
           .merge(players_df, how='left', on='nflId', validate='m:1')
           .merge(pff_scouting_df, how='left', on=['nflId', 'gameId', 'playId'], validate='m:1')
           .merge(plays_df, how='inner', on=['gameId', 'playId'], validate='m:1')
           )

#Generate the snap event dataframe
snap_df = gen_snap_df(week_df)
nonsnap_df = gen_nonsnap_df(week_df)

#Get the QB dataframe
passer_df = gen_passer_df(week_df)

#Get the pass blockers dataframe
pass_blockers_id_df, pass_blockers_poly_df = gen_line_df(week_df, position='pb')
all_pass_blockers_id_df, all_pass_blockers_poly_df = gen_line_df(week_df, position='pb', oline_pos_lineup=None)
pass_rushers_id_df, pass_rushers_poly_df = gen_line_df(week_df, position='pr')

In [8]:
pocket_area_df = gen_pocket_area_df(passer_df, pass_blockers_poly_df, snap_df, nonsnap_df)

In [9]:
pocket_area_df = pocket_area_df.drop([c for c in pocket_area_df.columns if 'pb_' in c], axis=1)
pocket_area_df = pocket_area_df.merge(pivot_line_df(all_pass_blockers_poly_df, 'pb'), how='left', on=['gameId', 'playId', 'frameId'])

pb_movement_cols = [c for c in pocket_area_df.columns if 'pb_movement_path' in c]
all_cols = list(chain(*[['gameId', 'playId', 'frameId', 'time', 'time_since_snap','snap_frameId','nonsnap_frameId', 'pocket_polygon', 'pocket_area'], pb_movement_cols]))

pocket_area_df = pocket_area_df[all_cols]
pocket_area_df = pocket_area_df.merge(pivot_line_df(pass_rushers_poly_df, 'pr')[['gameId', 'playId', 'frameId', 'pr_movement_path_0', 'pr_movement_path_1', 'pr_movement_path_2',
                                                       'pr_movement_path_3', 'pr_movement_path_4', 'pr_movement_path_5', 'pr_movement_path_6']], how='left', on=['gameId', 'playId', 'frameId'])


In [10]:
pocket_area_df['affected_pocket_polygon'] = pocket_area_df.apply(lambda x: gen_pocket_adj_risk_poly(x), axis=1)
pocket_area_df['affected_pocket_area'] = pocket_area_df.apply(lambda x: x['affected_pocket_polygon'].area, axis=1)

pocket_area_df['affected_pocket_area_pct'] = pocket_area_df['affected_pocket_area']/ pocket_area_df['pocket_area']

In [11]:
#Plot week append
week_df_append_polygon = pocket_area_df[['gameId', 'playId', 'frameId', 'pocket_polygon']]
week_df_append_polygon['team'] = 'pocket_polygon'

week_df_append_affected_polygon = pocket_area_df[['gameId', 'playId', 'frameId', 'affected_pocket_polygon']]
week_df_append_affected_polygon['team'] = 'affected_pocket_polygon'

#Create the appended data for plotting the polygons later
week_df_appended = pd.concat([week_df, week_df_append_affected_polygon, week_df_append_polygon], axis=0)

C:\Users\ben\AppData\Local\Temp\ipykernel_22488\2471944617.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ben\AppData\Local\Temp\ipykernel_22488\2471944617.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
def calc_pr_disruption_area(x):
    
    pb_paths = [p for p in x.index.values if 'pb_movement_path' in p]
    pb_paths_poly = [p for p in x[pb_paths].values if not(pd.isnull(p))]
    
    pb_combined = unary_union(pb_paths_poly)
    
    attack_poly = x['pr_movement_path'].symmetric_difference(pb_combined).difference(pb_combined)
    attack_poly = make_valid(attack_poly)
    
    attack_in_pocket_poly = make_valid(x['pocket_polygon']).intersection(attack_poly)
    return attack_in_pocket_poly

In [13]:
pocket_poly_df = pocket_area_df[['gameId', 'playId', 'frameId','snap_frameId','nonsnap_frameId', 'pocket_polygon', 'time_since_snap']]
pocket_poly_df = (pocket_poly_df
                  .merge(pivot_line_df(all_pass_blockers_poly_df,'pb'), how='left', on=['gameId', 'playId', 'frameId'])
                  .rename({'nflId': 'pb_nflId'}, axis=1)
                  .merge(pass_rushers_poly_df[['gameId', 'nflId', 'playId','frameId', 'pr_movement_path']], how='left', on=['gameId', 'playId', 'frameId'])
                  .rename({'nflId': 'pr_nflId'}, axis=1))
drop_cols = [c for c in pocket_poly_df.columns if 'pb_' in c and 'pb_movement_path' not in c]
pocket_poly_df = pocket_poly_df.drop(drop_cols, axis=1)
pocket_poly_df['pr_pocket_intersection'] = pocket_poly_df.apply(lambda x : calc_pr_disruption_area(x), axis=1)
pocket_poly_df.head()

KeyboardInterrupt: 

In [ ]:
pocket_poly_df['pocket_area'] = pocket_poly_df['pocket_polygon'].apply(lambda x: x.area)
pocket_poly_df['pr_pocket_intersection_area'] = pocket_poly_df['pr_pocket_intersection'].apply(lambda x: x.area)
pocket_poly_df['frame_counts'] = pocket_poly_df['nonsnap_frameId'] - pocket_poly_df['snap_frameId']
pocket_poly_df['frameId_rebased'] = pocket_poly_df['frameId'] - pocket_poly_df['snap_frameId']
haromonic_dict = {}
for i in range(int(pocket_poly_df['frameId_rebased'].max())):
    haromonic_dict[i] = nthHarmonic(i)

pocket_poly_df['max_weights'] = pocket_poly_df['frame_counts'].map(haromonic_dict)
pocket_poly_df['weights'] = pocket_poly_df['frameId_rebased'].map(haromonic_dict)
pocket_poly_df['inverse_weights'] = 1 - pocket_poly_df['weights'] / pocket_poly_df['max_weights']
pocket_poly_df['discount_factor'] = 1 / (1 + 0.1) ** (pocket_poly_df['frameId_rebased'] + 1)
pocket_poly_df['weighted_area'] = pocket_poly_df['discount_factor'] * (pocket_poly_df['pr_pocket_intersection_area'] / pocket_poly_df['pocket_area'])

In [ ]:
pocket_poly_df.head()

In [ ]:
def gen_area_stats(df):
    
    feat_dict = {}
    feat_dict['weighted_affected_area'] = np.sum(df['weighted_area'])
    feat_dict['time_until_penetration'] = df[df['weighted_area'] > 0]['time_since_snap'].min()
    feat_dict['max_affected_area'] = np.max(df['weighted_area'])
    return pd.Series(feat_dict)
    

In [ ]:
avg_pocket_disruption_df = pocket_poly_df.groupby(['gameId', 'playId', 'pr_nflId']).apply(lambda x: gen_area_stats(x)).reset_index()
avg_pocket_disruption_df.sort_values('weighted_affected_area', ascending=False, inplace=True)
avg_pocket_disruption_df = avg_pocket_disruption_df.merge(players_df[['nflId','displayName']], how='left', left_on='pr_nflId', right_on='nflId')
avg_pocket_disruption_df.head()

In [ ]:
avg_pocket_disruption_df.query('displayName == "Patrick Mahomes"')

In [ ]:
gameId = 2021092600
playId = 2879
pr_nflId = 52472.0

In [ ]:
# example_df = pocket_poly_df.query('(gameId == @gameId) & (playId == @playId) & (pr_nflId == @pr_nflId)')
example_df = pocket_poly_df.query('(gameId == @gameId) & (playId == @playId)')

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16,8))
sns.lineplot(example_df, x='frameId', y='discount_factor', ax=ax[0])
ax[0].set_title('Frame Weights')
sns.lineplot(example_df, x='frameId', y='weighted_area', ax=ax[1])
ax[1].set_title('Weighted Affected Area Score')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16,8))
sns.lineplot(example_df, x='frameId', y='inverse_weights', ax=ax[0])
ax[0].set_title('Frame Weights')
sns.lineplot(example_df, x='frameId', y='weighted_area', ax=ax[1])
ax[1].set_title('Weighted Affected Area Score')
plt.tight_layout()
plt.show()

In [ ]:
plot_game_play_id(week_df_appended, gameId, playId, size=(1500, 800))

In [ ]:
avg_pocket_disruption_df['time_until_penetration'] = np.where(pd.isnull(avg_pocket_disruption_df['time_until_penetration']), -1, avg_pocket_disruption_df['time_until_penetration'])
avg_pocket_disruption_df.head()

In [ ]:
play_ct_df = avg_pocket_disruption_df.groupby(['nflId','displayName'])['time_until_penetration'].count().reset_index().rename({'time_until_penetration': 'play_cts'}, axis=1)

In [ ]:
penetration_stats_df = (
    avg_pocket_disruption_df[avg_pocket_disruption_df['time_until_penetration'] > 0]
 .groupby(['nflId','displayName'])
 .apply(lambda x: 
        pd.Series({'time_until_penetration': np.mean(x['time_until_penetration']),
                   'non_zero_tup': len(x['time_until_penetration']),
                   'avg_max_affected_area': np.mean(x['max_affected_area'])}))
 .reset_index()
)
penetration_stats_df = penetration_stats_df.merge(play_ct_df, how='left', on=['nflId', 'displayName'])
penetration_stats_df = penetration_stats_df.assign(non_zero_tup_perc = lambda x: x['non_zero_tup'] / x['play_cts'])
penetration_stats_df.sort_values('non_zero_tup_perc', ascending=False, inplace=True)
penetration_stats_df = penetration_stats_df.query('play_cts > 30')
penetration_stats_df.head(10)

In [ ]:
import plotly.express as px
fig = px.scatter(penetration_stats_df, x="time_until_penetration", y="non_zero_tup_perc", hover_data=['displayName'])
fig.update_layout(width=1500, height=800)
fig.show()

## Lift Charts

In [ ]:
max_weighted_affected_area = avg_pocket_disruption_df.groupby(['gameId', 'playId'])['weighted_affected_area'].max().reset_index()
max_weighted_affected_area = max_weighted_affected_area.sort_values('weighted_affected_area', ascending=False)
max_weighted_affected_area = max_weighted_affected_area.merge(plays_df[['gameId', 'playId', 'passResult']], how='left', on=['gameId', 'playId'])
max_weighted_affected_area['pass_incomplete_ind'] = np.where(max_weighted_affected_area['passResult'].isin(['I', 'IN']), 1, 0)

gain_df = []
for i, df in enumerate(np.array_split(max_weighted_affected_area, 10)):
    df['decile'] = i
    gain_df.append(df)
gain_df = pd.concat(gain_df)
gain_df
# max_weighted_affected_area['decile'] = pd.qcut(max_weighted_affected_area['weighted_affected_area'], np.arange(0, 1.1, 0.1),labels=np.arange(1, 9, 1), duplicates='drop')

# max_weighted_affected_area.groupby('decile')['pass_incomplete_ind'].sum() 

# variable_score = max_weighted_affected_area.groupby('decile')['pass_incomplete_ind'].sum() / max_weighted_affected_area.groupby('decile')['pass_incomplete_ind'].count()

# baseline = max_weighted_affected_area['pass_incomplete_ind'].sum()/len(max_weighted_affected_area)
# fig, ax = plt.subplots(1,1, figsize=(12,8))
# sns.lineplot(data=variable_score.reset_index(), x='decile', y='pass_incomplete_ind', ax=ax, marker="o")
# ax.axhline(baseline, linestyle='--', alpha=0.5)
# ax.set_ylabel('Pass Incompletion Percentage')
# ax.set_xlabel('Pocket Disruption Score Decile')
# plt.show()

In [ ]:
gain_chart_df = (
    gain_df
     .groupby('decile')
     .apply(lambda x: 
            pd.Series({
                'no_of_plays': len(x),
                'pass_inc_ct': np.sum(x['pass_incomplete_ind'])})
           )
     .reset_index()
     .assign(
         decile = lambda x: x['decile'] + 1
     )
)

gain_chart_df.loc[-1] = [0, 0, 0]
gain_chart_df.index = gain_chart_df.index + 1
gain_chart_df.sort_index(inplace=True) 

gain_chart_df = (
    gain_chart_df
    .assign(
         cum_pass_inc_ct = lambda x: x['pass_inc_ct'].cumsum(),
         perc_of_events = lambda x: x['pass_inc_ct'] / x['pass_inc_ct'].sum() * 100,
         perc_of_cumulative_plays = lambda x: x['no_of_plays'].cumsum() / x['no_of_plays'].sum() * 100
     )
     .assign(
         gain = lambda x: x['cum_pass_inc_ct'] / x['pass_inc_ct'].sum() * 100
     )
     .assign(
         cumulative_lift = lambda x: x['gain'] / (10*x['decile'])
     )
)

gain_chart_df

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))
sns.lineplot(data=gain_chart_df, x='decile', y='gain', ax=ax, marker="o")
sns.lineplot(data=gain_chart_df, x='decile', y='perc_of_cumulative_plays', ax=ax, marker="o")
ax.set_ylabel('Pass Incompletion Percentage')
ax.set_xlabel('Pocket Disruption Score Decile')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,8))
sns.lineplot(data=gain_chart_df.dropna(), x='perc_of_cumulative_plays', y='cumulative_lift', ax=ax, marker="o")
sns.lineplot(data=gain_chart_df.dropna(), x='perc_of_cumulative_plays', y=1, ax=ax, marker="o")
ax.set_ylabel('Pass Incompletion Percentage')
ax.set_xlabel('Pocket Disruption Score Decile')
plt.show()

In [ ]:
from sklearn import metrics
from scipy import stats

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(gain_df['pass_incomplete_ind'], gain_df['weighted_affected_area'], pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
stats.pearsonr(gain_df['pass_incomplete_ind'], gain_df['weighted_affected_area'])

In [ ]:
pff_scouting_df.head()

In [ ]:
pff_scouting_df['pff_hurry'].fillna(0)

In [ ]:
week_df_baseline = week_df.copy(deep=True)
week_df_baseline['pff_hurry'] = week_df_baseline['pff_hurry'].fillna(0)
week_df_baseline = week_df_baseline.groupby(['gameId', 'playId', 'passResult'])['pff_hurry'].max().reset_index()
week_df_baseline['pass_incomplete_ind'] = np.where(week_df_baseline['passResult'].isin(['I', 'IN']), 1, 0)
week_df_baseline

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(week_df_baseline['pass_incomplete_ind'], week_df_baseline['pff_hurry'], pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
stats.pearsonr(week_df_baseline['pass_incomplete_ind'], week_df_baseline['pff_hurry'])

In [ ]:
0.574 / 0.55

In [ ]:
plays_df['dropBackType'].unique()